# Grouping and Aggregating Data

## Introduction
This notebook demonstrates how to group and aggregate data to gain insights into customer behavior for the Instacart dataset. The goal is to explore trends, understand spending habits, and analyze ordering patterns.

#### Table of Contents
1. Import Libraries and Load Data  
2. Group and Aggregate Data  
3. Loyalty Flags for Existing Customers
    - 3.1 Inspect examples from each loyalty category
        - Examples of Loyal Customers
        - Examples of Regular Customers
        - Examples of New Customers
4. Analyzing Spending Habits  
5. Handle Outliers in Prices
6. Spending Flags
    - 6.1 Analyze Spending Flag Distribution
7. Order Frequency Flags
8. Analyze the Distribution of Order Frequency Flags
9. Summary
10. Export Data 

## 1. Import Libraries and Load Data

In [126]:
# Importing required libraries
import pandas as pd
import numpy as np

# Load the merged_orders_products pickle file
merged_orders_products = pd.read_pickle(
    r"D:\YVC\Data Analytics (CF)\Python Fundamentals for Data Analysts\Instacart Basket Analysis\02 Data\Prepared Data\merged_orders_products_updated.pkl"
)

# Display the first few rows to confirm the file loaded correctly
print(merged_orders_products.head())

   order_id  product_id  add_to_cart_order  reordered           product_name  \
0         2       33120                  1          1     Organic Egg Whites   
1         2       28985                  2          1  Michigan Organic Kale   
2         2        9327                  3          0          Garlic Powder   
3         2       45918                  4          1         Coconut Butter   
4         2       30035                  5          0      Natural Sweetener   

   aisle_id  department_id  prices        price_label  order_dow  \
0        86             16    11.3  Mid-range product          5   
1        83              4    13.4  Mid-range product          5   
2       104             13     3.6  Low-range product          5   
3        19             13     8.4  Mid-range product          5   
4        17             13    13.7  Mid-range product          5   

      busiest_day  order_hour_of_day_x  order_hour_of_day_y  \
0  Regularly busy                    9         

#### Load the Orders Products Combined Dataset

In [56]:
# Load the orders_products_combined dataset
orders_products_combined = pd.read_pickle(
    r"D:\YVC\Data Analytics (CF)\Python Fundamentals for Data Analysts\Instacart Basket Analysis\02 Data\Original Data\orders_products_combined.pkl"
)

# Display the first few rows to confirm successful loading
print(orders_products_combined.head())

   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2539329        1    prior             1          2                  8   
2   2539329        1    prior             1          2                  8   
3   2539329        1    prior             1          2                  8   
4   2539329        1    prior             1          2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     NaN         196                  1          0   
1                     NaN       14084                  2          0   
2                     NaN       12427                  3          0   
3                     NaN       26088                  4          0   
4                     NaN       26405                  5          0   

                              product_name  aisle_id  department_id  prices  \
0                              

## 2. Group and Aggregate Data
Here, I’m grouping the data by `department_id` and calculating the average `order_number` to spot ordering patterns across different departments.

In [60]:
# Grouping by department_id and calculating the mean of order_number
department_order_mean = orders_products_combined.groupby('department_id')['order_number'].mean()

# Display the result
print(department_order_mean)

department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.213779
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170828
12    15.887622
13    16.583304
14    16.757377
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


#### Observations
- **Departments 21 (22.90) and 10 (20.20)** have the highest average order numbers. This suggests that people are frequently coming back to reorder products from these departments.
- **Departments 1 (15.46) and 5 (15.21)** have the lowest average order numbers, indicating that products in these categories might not be reordered as often.
- **The average order numbers for departments range from 15.21 to 22.90,** showing significant variation in reorder frequency depending on the department.

## 3. Loyalty Flags for Existing Customers
Here, we'll create loyalty flags to classify customers based on their total orders.

In [162]:
# Step 1: Calculate the total number of orders per user
customer_orders = orders_products_combined.groupby('user_id')['order_number'].max().reset_index()
customer_orders.rename(columns={'order_number': 'total_orders'}, inplace=True)

# Step 2: Merge total orders back into the dataset
# Handle potential column conflicts by renaming or dropping duplicates
orders_products_combined = orders_products_combined.drop(columns=['total_orders'], errors='ignore')  # Avoid duplicate column error
orders_products_combined = orders_products_combined.merge(customer_orders, on='user_id', how='left')

# Step 3: Create a loyalty flag
orders_products_combined['loyalty_flag'] = 'New Customer'
orders_products_combined.loc[orders_products_combined['total_orders'] >= 40, 'loyalty_flag'] = 'Loyal Customer'
orders_products_combined.loc[
    (orders_products_combined['total_orders'] >= 10) & (orders_products_combined['total_orders'] < 40),
    'loyalty_flag'
] = 'Regular Customer'

# Step 4: Verify results
# Check unique values of total_orders
print("Unique Values in Total Orders:")
print(customer_orders['total_orders'].describe())

# Check the distribution of loyalty flags
print("\nLoyalty Flag Distribution:")
print(orders_products_combined['loyalty_flag'].value_counts())

Unique Values in Total Orders:
count    206209.000000
mean         15.590367
std          16.654774
min           3.000000
25%           5.000000
50%           9.000000
75%          19.000000
max          99.000000
Name: total_orders, dtype: float64

Loyalty Flag Distribution:
loyalty_flag
Regular Customer    16297258
Loyal Customer      10661369
New Customer         5475585
Name: count, dtype: int64


### 3.1 Inspect examples from each loyalty category

**Examples of Loyal Customers:**

In [166]:
# Inspect examples from each loyalty category
print("\nExamples of Loyal Customers:")
print(orders_products_combined[orders_products_combined['loyalty_flag'] == 'Loyal Customer'].head())


Examples of Loyal Customers:
      order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
1486   1737705       17    prior             1          2                 13   
1487   1737705       17    prior             1          2                 13   
1488   1737705       17    prior             1          2                 13   
1489   1681401       17    prior             2          5                 10   
1490   1681401       17    prior             2          5                 10   

      days_since_prior_order  product_id  add_to_cart_order  reordered  \
1486                     NaN        7350                  1          0   
1487                     NaN       47141                  2          0   
1488                     NaN       17762                  3          0   
1489                     3.0        7350                  1          1   
1490                     3.0       18534                  2          0   

                                     product

**Examples of Regular Customers:**

In [170]:
print("\nExamples of Regular Customers:")
print(orders_products_combined[orders_products_combined['loyalty_flag'] == 'Regular Customer'].head())


Examples of Regular Customers:
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2539329        1    prior             1          2                  8   
2   2539329        1    prior             1          2                  8   
3   2539329        1    prior             1          2                  8   
4   2539329        1    prior             1          2                  8   

   days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                     NaN         196                  1          0   
1                     NaN       14084                  2          0   
2                     NaN       12427                  3          0   
3                     NaN       26088                  4          0   
4                     NaN       26405                  5          0   

                              product_name  aisle_id  department_id  prices  \

**Examples of New Customers:**

In [175]:
print("\nExamples of New Customers:")
print(orders_products_combined[orders_products_combined['loyalty_flag'] == 'New Customer'].head())


Examples of New Customers:
     order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
342   3343014        4    prior             1          6                 11   
343   3343014        4    prior             1          6                 11   
344   3343014        4    prior             1          6                 11   
345   3343014        4    prior             1          6                 11   
346   2030307        4    prior             2          4                 11   

     days_since_prior_order  product_id  add_to_cart_order  reordered  \
342                     NaN       36606                  1          0   
343                     NaN        7350                  2          0   
344                     NaN       35469                  3          0   
345                     NaN        2707                  4          0   
346                    19.0       42329                  1          0   

                                          product_name  ai

#### **Key Insights:**

**1. Loyal Customers:**
\
~ Customers with total orders >= 40 are flagged as "Loyal Customers."

**2. Regular Customers:**
\
~ Customers with total orders between 10 and 39 are flagged as "Regular Customers."

**3. New Customers:**
\
~ Customers with total orders < 10 are flagged as "New Customers."

## 4. Analyzing Spending Habits
Let's take a closer look and compare at how spending habits differ among the loyalty groups.

In [81]:
# Analyze spending habits by loyalty group
spending_stats = orders_products_combined.groupby('loyalty_flag')['prices'].agg(['mean', 'median', 'min', 'max'])

# Display the results
print(spending_stats)

                       mean  median  min      max
loyalty_flag                                     
Loyal Customer    10.379575     7.4  1.0  99999.0
New Customer      13.208206     7.4  1.0  99999.0
Regular Customer  12.616590     7.4  1.0  99999.0


#### **Key Insights:**

**1. Mean Spending:**
\
~ Loyal Customers spend an average of **10.38** per product.
\
~ Regular Customers spend an average of **12.62** per product.
\
~ New Customers spend the most, with an average of **13.20** per product.

**2. Median Spending:**
\
~ All groups share the same median price of **7.4**, indicating a significant proportion of products fall into this price range.

**3. Minimum Prices:**
\
~ All groups have a minimum product price of **1.0**, reflecting purchases of low-cost items across all loyalty segments.

**4. Maximum Prices:**
\
~ All groups share the same maximum product price of **99,999**, which likely represents an outlier or data entry error.


## 5. Handle Outliers in prices
To clean up the dataset, we'll filter out outliers in the `prices` column.

In [85]:
# Remove outliers where prices are extremely high
filtered_data = orders_products_combined[orders_products_combined['prices'] < 100]

# Recalculate spending stats without outliers
filtered_spending_stats = filtered_data.groupby('loyalty_flag')['prices'].agg(['mean', 'median', 'min', 'max'])

# Display the updated results
print(filtered_spending_stats)

                      mean  median  min   max
loyalty_flag                                 
Loyal Customer    7.778456     7.4  1.0  25.0
New Customer      7.804438     7.4  1.0  25.0
Regular Customer  7.804054     7.4  1.0  25.0


#### Observations After Removing Outliers

**1. Mean Spending:**
\
~ After removing outliers, the average spending across all loyalty groups is pretty consistent, ranging between **7.78** and **7.80**.

**2. Median Spending:**
\
~ The median price stays the same at **7.4** for all groups, which shows that most purchases fall into this price range.

**3. Minimum Prices:**
\
~ Every group has a minimum price of **1.0**, which means customers across all segments are buying low-cost items.

**4. Maximum Prices:**
\
~ The highest price is now **25.0** after filtering out outliers, which makes the data more realistic and easier to analyze.

**5. Outlier Handling:**
\
~ By removing extreme prices over 100, the spending data now feels much cleaner and gives a clearer picture of how customers are actually spending.


## 6. Spending Flag
In this section, we’re creating a spending flag to classify customers based on their average spending. This will help us better understand which customers are spending more or less on their purchases.

In [90]:
# Step 1: Calculate average price per user
user_spending = filtered_data.groupby('user_id')['prices'].mean().reset_index()
user_spending.rename(columns={'prices': 'avg_price'}, inplace=True)

# Step 2: Merge average price back into the dataset
filtered_data = filtered_data.merge(user_spending, on='user_id', how='left')

# Step 3: Create a spending flag
filtered_data['spending_flag'] = 'Low Spender'
filtered_data.loc[filtered_data['avg_price'] >= 10, 'spending_flag'] = 'High Spender'

# Check the results
print(filtered_data[['user_id', 'avg_price', 'spending_flag']].head(10))

   user_id  avg_price spending_flag
0        1   6.367797   Low Spender
1        1   6.367797   Low Spender
2        1   6.367797   Low Spender
3        1   6.367797   Low Spender
4        1   6.367797   Low Spender
5        1   6.367797   Low Spender
6        1   6.367797   Low Spender
7        1   6.367797   Low Spender
8        1   6.367797   Low Spender
9        1   6.367797   Low Spender


#### **Key Insights:**

**1. Spending Categories:**
~ All customers are classified as **Low Spenders**, meaning their average spending per product is below 10.

**2. Spending Patterns:**
~ The lack of **High Spenders** indicates that customers predominantly purchase lower-priced items, suggesting that budget-friendly products dominate the dataset.

**3. Implications for Targeting:**
~ Since there are no **High Spenders**, marketing efforts could focus on promoting affordable products or offering discounts to encourage more purchases from existing customers.

### 6.1 Analyze Spending Flag Distribution

In [93]:
# Analyze the distribution of spending flags
spending_flag_counts = filtered_data['spending_flag'].value_counts()

# Display the results
print(spending_flag_counts)

spending_flag
Low Spender     32308374
High Spender      120711
Name: count, dtype: int64


#### **Key Findings:**

1. **Low Spenders**:
\
   ~ There are **3,328,374 users** (~96.5% of the dataset) in this category.
   \
   ~ This indicates that most customers are spending less than $10 on average.

3. **High Spenders**:
   \
   ~ Only **120,711 users** (~3.5% of the dataset) fall into this category.
   \
   ~ These customers spend more than or equal to $10 on average.

5. **Insights for Targeting**:
\
   ~ Since the majority are Low Spenders, marketing strategies could focus on offering discounts or bundles for budget-conscious shoppers.
\
   ~ High Spenders, though a smaller group, might respond well to personalized premium product recommendations or loyalty rewards.

## 7. Order Frequency Flag
In this section, I’m using the `days_since_prior_order` column to create an order frequency flag. This will help categorize customers based on how often they place orders.

In [101]:
# Step 1: Calculate median days_since_prior_order per user
user_frequency = filtered_data.groupby('user_id')['days_since_prior_order'].median().reset_index()
user_frequency.rename(columns={'days_since_prior_order': 'median_days_since_order'}, inplace=True)

# Step 2: Merge median days_since_prior_order back into the dataset
filtered_data = filtered_data.merge(user_frequency, on='user_id', how='left')

# Step 3: Create an order frequency flag
filtered_data['order_frequency_flag'] = 'Non-Frequent Customer'
filtered_data.loc[filtered_data['median_days_since_order'] <= 20, 'order_frequency_flag'] = 'Regular Customer'
filtered_data.loc[filtered_data['median_days_since_order'] <= 10, 'order_frequency_flag'] = 'Frequent Customer'

# Check the results
print(filtered_data[['user_id', 'median_days_since_order', 'order_frequency_flag']].head(10))

   user_id  median_days_since_order   order_frequency_flag
0        1                     20.5  Non-Frequent Customer
1        1                     20.5  Non-Frequent Customer
2        1                     20.5  Non-Frequent Customer
3        1                     20.5  Non-Frequent Customer
4        1                     20.5  Non-Frequent Customer
5        1                     20.5  Non-Frequent Customer
6        1                     20.5  Non-Frequent Customer
7        1                     20.5  Non-Frequent Customer
8        1                     20.5  Non-Frequent Customer
9        1                     20.5  Non-Frequent Customer


#### **Key Insights:**

**1. Order Frequency Categories:**
\
~ Customers are divided into three groups based on their ordering behavior:
  - **Non-Frequent Customer**: Median days since prior order > 20.
  - **Regular Customer**: Median days between 10 and 20.
  - **Frequent Customer**: Median days <= 10.

**2. Observations from the Data:**
\
~ Most customers fall into the **Non-Frequent Customer** category, indicating they tend to place orders less frequently.
\
~ **Regular Customers** and **Frequent Customers** make up a smaller portion of the dataset.

**3. Implications for Targeting:**
\
~ **Frequent Customers** could be targeted with loyalty programs or incentives to maintain their high engagement.
\
~ **Regular Customers** might benefit from personalized reminders to encourage more frequent purchases.
\
~ **Non-Frequent Customers** may respond well to discounts or promotions to bring them back into the platform.

## 8. Analyze the Distribution of Order Frequency Flags
Here’s how customers are distributed based on their ordering frequency:

- **Frequent Customers**: 21,574,198 users (~62.3% of the dataset), meaning most customers place orders regularly, within 10 days.
- **Regular Customers**: 7,215,912 users (~20.8% of the dataset), with orders placed every 10 to 20 days.
- **Non-Frequent Customers**: 3,638,975 users (~16.9% of the dataset), showing that a smaller portion of customers take longer than 20 days to reorder.

These insights give a clear picture of how often customers are engaging with the platform, which can be used to tailor marketing strategies for each group.

In [104]:
# Analyze the distribution of order frequency flags
order_frequency_counts = filtered_data['order_frequency_flag'].value_counts()

# Display the results
print(order_frequency_counts)

order_frequency_flag
Frequent Customer        21574198
Regular Customer          7215912
Non-Frequent Customer     3638975
Name: count, dtype: int64


#### **Key Insights:**

1. **Frequent Customers**:
\
   ~ This is the largest group (~62.3%) and represents customers who engage most actively with the platform.
\
   ~ Strategies like loyalty rewards could be effective to maintain their high engagement.

3. **Regular Customers**:
\
   ~ This group (~20.8%) orders less frequently but still consistently.
\
   ~ Personalized recommendations or reminders might encourage them to order more often.

5. **Non-Frequent Customers**:
\
   ~ The smallest group (~16.9%) represents customers who are less engaged.
\
   ~ Targeted discounts or re-engagement campaigns could help bring them back to the platform.

7. **Actionable Insights**:
\
   ~ These distributions provide a solid foundation for segmenting and tailoring marketing strategies to boost customer engagement across all groups.

## 9. Summary

In this task, I focused on grouping and aggregating data from the Instacart dataset to better understand customer behavior. Here’s what I worked on:

1. **Grouping and Aggregating Data**:
\
   ~ I analyzed ordering patterns across departments by calculating the average order number.

3. **Loyalty Flags**:
\
   ~ I classified customers into three groups: **Loyal Customers**, **Regular Customers**, and **New Customers**, based on their total number of orders.

4. **Spending Habits**:
\
   ~ I looked into spending patterns across loyalty groups. Most customers were identified as low spenders, with only a small portion being high spenders.

5. **Spending Flags**:
\
   ~ I created flags to classify users as **Low Spenders** or **High Spenders** based on their average spending. Almost all customers were categorized as low spenders.

6. **Order Frequency Flags**:
\
   ~ I categorized customers into **Frequent**, **Regular**, or **Non-Frequent Customers** using the median `days_since_prior_order`. Most users were frequent customers, placing orders within 10 days.

7. **Key Insights**:
\
   ~ These findings help highlight customer behavior and provide valuable insights for targeted marketing strategies. For example, Instacart could focus on budget-friendly promotions for low spenders or personalized loyalty programs for frequent customers.

This notebook is polished, structured, and ready for use in the final project report.

## 10. Export Data

In [322]:
# Export the dataset
merged_orders_products.to_pickle(
    r"D:\YVC\Data Analytics (CF)\Python Fundamentals for Data Analysts\Instacart Basket Analysis\02 Data\Prepared Data\4.8_updated_orders_products.pkl"
)